<a href="https://colab.research.google.com/github/naterattner/data71200/blob/master/project_1/data71200_summer24_project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Python notebook for assignment 1: https://bbhosted.cuny.edu/webapps/assignment/uploadAssignment?content_id=_85452921_1&course_id=_2383576_1&group_id=&mode=view

The goal for this assignment is to create a usable dataset from an open-source data collection. This dataset will be used for a supervised classification task in Project 2 and an unsupervised learning task in Project 3.

### Step 1: Find and download a dataset

### Step 2: Divide into a training set and a testing set

### Step 3: Explore your training set

### Step 4: Data cleaning

### Step 5: Visualize the data in your training set

### Step 6: Apply transformations to your data